# DeepLabCut Toolbox - Colab
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:
- train a network
- evaluate a network
- analyze a novel video

This assumes you already have a project folder with labeled data! 


## Let's look at info about the Colab Environment:


In [0]:
!nvcc --version
import tensorflow as tf
tf.__version__
tf.test.gpu_device_name()
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [0]:
#Colab only - to link to your GoogleDrive, run this cell and follow the authorization instructions:
#(We recommend putting a copy of github repo as a folder in your google drive)

from google.colab import drive
drive.mount('/content/drive')

In [0]:
#--# NEW - CHANGE

#in colab: go to your deeplabcut folder --- TODO - get 'better' deeplabcut repo
%cd /content/drive/My Drive/ColabFiles/qbio

#You'll need to uncomment a few lines of code to make the notebook work:
#(this will take a few minutes to install all the dependences!)

!pip install deeplabcut
!pip install --no-cache-dir -I pillow
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [0]:
#GUIs don't work on the cloud, so label your data locally on your computer! 
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

In [0]:
import deeplabcut

#sometimes the backend kernel is defaulting to something other than what we want
#if you see the warnings, you can re-run this cell and they go way, as we re-set the backend! :) 

#possible warning--> UserWarning: This call to matplotlib.use() has no effect because the backend has already
#been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot, or matplotlib.backends is imported for the first time.

In [0]:
#--# NEW - edit path -- in project.config.yaml
#--# RESTART - edit -- dlc-models/../train/pose_cfg.yaml

#--# NEW - edit path
path_config_file = '/content/drive/My Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/config.yaml'


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!


## Create a training dataset:
### we recommend doing this outside of Colab, then copying your file into Drive
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [0]:
#If you didn't run this already...
#And if you DID, you may get errors, as you need to delete the created folder first - they will not be overwritten! (i.e. delete "dlc-models")
#deeplabcut.create_training_dataset(path_config_file,Shuffles=[1])

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [0]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more infor, and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100,saveiters=5000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['head', 'tail', 'up', 'down'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_SwimmingJan25/Swimming_Shivansh95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/content/drive/My '
                 'Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/dlc-models/iteration-0/SwimmingJan25-trainset95shuffle1/train/snapshot-175000',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1000,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_SwimmingJan25/Documentation_data-Swimming_95sh

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/dlc-models/iteration-0/SwimmingJan25-trainset95shuffle1/train/snapshot-175000


Restoring parameters from /content/drive/My Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/dlc-models/iteration-0/SwimmingJan25-trainset95shuffle1/train/snapshot-175000
Restoring parameters from /content/drive/My Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/dlc-models/iteration-0/SwimmingJan25-trainset95shuffle1/train/snapshot-175000


Display_iters overwritten as 100
Save_iters overwritten as 5000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/ColabFiles/qbio/track-exp5-6__Swimming-Shivansh-2019-01-25/dlc-models/iteration-0/SwimmingJan25-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, '

iteration: 100 loss: 0.0322 lr: 0.005
iteration: 100 loss: 0.0322 lr: 0.005
iteration: 200 loss: 0.0092 lr: 0.005
iteration: 200 loss: 0.0092 lr: 0.005


**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [0]:
#this is a colab specific work-around, but it works!

!pip install Pillow==4.0.0

from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

In [0]:
%matplotlib inline

deeplabcut.evaluate_network(path_config_file,Shuffles=[1],plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- pplease see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [0]:
videofile_path = ['/content/drive/My Drive/DeepLabCut/examples/Reaching-Mackenzie-2018-08-30/videos/MovieS2_Perturbation_noLaser_compressed.avi'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
#for making interactive plots.
%matplotlib notebook 
deeplabcut.plot_trajectories(path_config_file,videofile_path)